In [1]:
import pandas as pd
import numpy as np
import os

### Cache Series Data

In [2]:
dirList = os.listdir("data/LargerTransformed/")
fList = []
for filename in dirList: #limit to .csv
    if(filename[-4:] == '.csv'):
        fList.append(filename[:-4]) 

In [32]:
dfs = []
count = 0
for fName in fList[:]:
    fContent = pd.read_csv('data/LargerTransformed/'+fName+'.csv')
#     fContent.columns = ['ContestId', 'SecondsRemaining', 'Entries']
    count = count+len(fContent)
    dfs.append(fContent)
seriesData = pd.concat(dfs).set_index('ContestId')


In [33]:
contests = pd.read_csv('data/WorkingData.csv').set_index('ContestId')[['TotalPrizeAmount', 'EntryFeeAmount']]
contests = contests.assign(SuccessThreshold= lambda x:(contests['TotalPrizeAmount'].astype(float)/(contests['EntryFeeAmount'].astype(float))).astype(int))

In [47]:
combinedDF = pd.merge(contests[['SuccessThreshold']], seriesData, on='ContestId', how='right').dropna()

In [48]:
combinedDF.head()

,SuccessThreshold,0HoursOut,10HoursOut,12HoursOut,16HoursOut,1HoursOut,20HoursOut,24HoursOut,2HoursOut,36HoursOut,3HoursOut,48HoursOut,4HoursOut,5HoursOut,6HoursOut,7HoursOut,8HoursOut,9HoursOut
ContestId,,,,,,,,,,,,,,,,,,
24567084,50.0,39,17,15,13,33,10,10,29,9,25,8,22,22,22,22,21,18
24567089,200.0,235,187,166,145,235,135,127,235,114,231,95,226,225,222,215,208,195
24567090,33.0,18,4,3,3,15,2,2,15,2,11,2,8,8,7,7,7,6
24567091,15.0,10,5,5,5,10,5,5,10,3,8,3,6,6,6,6,6,5
24567093,360.0,182,123,110,101,170,96,94,153,92,151,80,141,135,135,135,127,123


In [49]:
scaleFeatures = combinedDF.columns.drop('SuccessThreshold')
for f in scaleFeatures:
    combinedDF = combinedDF.assign(newCol= lambda x:(combinedDF[f].astype(float)/(combinedDF['SuccessThreshold'].astype(float))).astype(float))
    combinedDF = combinedDF.rename(columns={'newCol': f+'_scaled'}).drop(columns=[f])
                                            

In [52]:
hoursOutDf = combinedDF.drop(columns='SuccessThreshold')

In [54]:
workedDF = pd.read_csv('data/WorkedData.csv').set_index('ContestId')


In [59]:
combinedDF = pd.merge(workedDF, hoursOutDf, on='ContestId', how='right').dropna()
combinedDF.head()

,Success,SportName_NFL,SportName_PGA,SportName_SOCC,SportName_MLB,SportName_NAS,SportName_CFB,SportName_MMA,SportName_LOL,SportName_NHL,...,2HoursOut_scaled,36HoursOut_scaled,3HoursOut_scaled,48HoursOut_scaled,4HoursOut_scaled,5HoursOut_scaled,6HoursOut_scaled,7HoursOut_scaled,8HoursOut_scaled,9HoursOut_scaled
ContestId,,,,,,,,,,,,,,,,,,,,,
24567084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.580000,0.180000,0.500000,0.160000,0.440000,0.440000,0.440000,0.440000,0.420000,0.360000
24567089,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.175000,0.570000,1.155000,0.475000,1.130000,1.125000,1.110000,1.075000,1.040000,0.975000
24567090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.454545,0.060606,0.333333,0.060606,0.242424,0.242424,0.212121,0.212121,0.212121,0.181818
24567091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.666667,0.200000,0.533333,0.200000,0.400000,0.400000,0.400000,0.400000,0.400000,0.333333
24567093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.425000,0.255556,0.419444,0.222222,0.391667,0.375000,0.375000,0.375000,0.352778,0.341667


In [60]:
combinedDF.to_csv('data/SCALEDEVERYTHING.csv')